# DataMadness
Group: 20  
Date: 24/03/2021  
Members:
- Stijn Coppens, i6158177
- Tijn Logtens, i......
- Luc Sparidans, i6166245

## Table of Contents
- Imports / Setup
- Question 1
- Question 2
- Question 3
    - Scraping
    - Data cleaning
    - Analysis
- Question 4

## Imports / Setup

In [ ]:
# imports and setup

import scipy as sc
import numpy as np

import pandas as pd
import statsmodels.formula.api as sm
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

from IPython.display import Video

import matplotlib.pyplot as plt
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10, 6)

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import yfinance as yf
from bs4 import BeautifulSoup
import urllib.request
import datetime

## Question 1

## Question 2

## Question 3

### 3.1 Scraping

In [ ]:
url = "http://www.espn.com/nfl/superbowl/history/winners"

req = urllib.request.Request(url)
with urllib.request.urlopen(req) as response:
    html = response.read()

class_soup = BeautifulSoup(html, 'html.parser')

# we can retreive all tables, our desired table is the first one:
table_html = class_soup("table")[0]

# converting to pandas dataframe
dates = pd.read_html(str(table_html), header=0)[0]

# selecting only the Year and Date column
dates['Year'] = dates['Super Bowl Winners and Results.1']
dates['Date'] = dates['Super Bowl Winners and Results.1']
dates = dates.drop(['Super Bowl Winners and Results','Super Bowl Winners and Results.1','Super Bowl Winners and Results.2','Super Bowl Winners and Results.3'], axis=1)
dates = dates.drop(0)

# maping the Year column to only year 
# make true if you want int instead of string
inte = True
def year(year):
    tmp = year.split()
    if inte:
        return int(tmp[2])
    else:
        return tmp[2]
    
dates['Year'] = dates['Year'].map(year)
dates = dates.loc[dates['Year'] > 1999]
dates = dates.loc[dates['Year'] < 2021]

# Maping the Date column to datetime datatype
def date(date):
    tmp = date.split()
    tmp1 = int(tmp[2])
    tmp2 = tmp[1]
    tmp2 = int(tmp2[:len(tmp[1])-1])
    if 'Jan' in tmp[0]:
        return datetime.datetime(tmp1,1,tmp2)
    else:
        return datetime.datetime(tmp1,2,tmp2)

dates['Date'] = dates['Date'].map(date)

# Reading the superbowl ads csv and selecting every brand
name = pd.read_csv("DataSets/superbowl-ads.csv")
name = name.drop_duplicates(subset=['brand'])

# constructing a series with the right information
d = ['TM','BUD','HYMTF','KO','KIMTF','BUD',' ','PEP','PEP','ETFC']
stock_name = pd.Series(data=d, index=name['brand'])

In [ ]:
COMPANIES = ['TM','BUD','HYMTF','KO','KIMTF','PEP']
all_data = pd.DataFrame()
for NAME in COMPANIES:
    stock = yf.Ticker(NAME)
    df = stock.history(start="2000-01-01",end="2021-01-01",interval="1mo")
    df = df.dropna()
    df = df.drop(['Dividends','Stock Splits'],axis=1)
    avg_price = (df['Low'] + df['High']) / 2
    all_data[NAME] = avg_price

In [ ]:
tmp = []
for year in dates['Year']:
    tmp.append(all_data.loc[''.join((str(year),'-03-01'))]-all_data.loc[''.join((str(year),'-01-01'))])

delta = pd.DataFrame(tmp)
delta = delta.set_index(dates['Year'])
delta

**Note:** Dropped E-Trade and NFL since they do not have valid stock information listed on Yahoo Finance.

### 3.2 Data Cleaning

In [ ]:
all_data.plot()
delta.plot()

In [ ]:
ad = pd.read_csv('../DataMadness/DataSets/superbowl-ads(sorted).csv')
ad = ad.drop(['superbowl_ads_dot_com_url','youtube_url'],axis=1)
ad = ad.set_index('year')
ad = ad[(ad['brand'] != 'E-Trade') & (ad['brand'] != 'NFL')]
ad

In [ ]:
name = ad.drop_duplicates(subset=['brand'])
d = ['BUD','BUD','PEP','PEP','TM','KO','HYMTF','KIMTF']
stock_name = pd.Series(data=d, index=name['brand'])
def stock(name):
    return stock_name.loc[name]
ad['stock_name'] = ad['brand'].map(stock)
ad

In [ ]:
price = []
for index, row in ad.iterrows():
    price.append(delta.loc[index, row['stock_name']])
ad['price'] = price
ad = ad.dropna()
brands = ad.groupby(['brand'])
plt.figure()

# Iterate through continents

for i, (brand_name, price) in enumerate(brands):
    # create subplot axes in a 3x3 grid
    ax = plt.subplot(3, 3, i + 1) # nrows, ncols, axes position
    # plot the continent on these axes
    price.plot(ax=ax)
    # set the title
    ax.set_title(brand_name)

plt.tight_layout()
plt.show()


### 3.3 Analysis

In [ ]:
SLR = sm.ols("price ~ celebrity + danger + use_sex", data=ad).fit()
SLR.summary()

In [ ]:
ad_no_kia = ad[ad['brand'] != 'Kia']
nk_r = sm.ols("price ~ celebrity + danger + use_sex", data=ad_no_kia).fit()
nk_r.summary()

In [ ]:
brnds = ad[['brand','price']].groupby(['brand'])

fig, ax = plt.subplots(figsize=(8,6))
for label, df in brnds:
    df.price.plot(ax=ax, label=label)
plt.legend()

## Question 4

In [ ]:
grp = ad.groupby('brand')[['funny', 'show_product_quickly', 'patriotic', 'celebrity', 'danger', 'animals', 'use_sex']]
tmp = pd.DataFrame(grp.sum())
size = grp.size()
for index, row in tmp.iterrows():
    tmp.loc[index] = row.divide(size.loc[index])
tmp